In [ ]:
import torch
import torchvision.transforms as transforms

# custom imports
from FTDDataset.FTDDataset import FloorTypeDetectionDataset
from models.unimodal_models import LeNet_Like, VGG_Like
from models.multimodal_models import LeNet_Like_multimodal
from train import Trainer
from eval import evaluate, load_state_dict
from visualization.visualization import visualize_data_sample_or_batch
from custom_utils.custom_utils import gen_run_dir, start_logger, store_used_config
import os

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# variables for dataset and config to use
dataset_path = r"C:\Users\Dominik\Downloads\FTDD_1.0"
mapping_filename = "label_mapping_full_dataset.json"
preprocessing_config_filename = "preprocessing_config.json"

# list of sensors to use
# sensors = ['accelerometer', 'BellyCamLeft', 'BellyCamRight', 'bodyHeight', 'ChinCamLeft', 'ChinCamRight', 'footForce', 'gyroscope',
#            'HeadCamLeft', 'HeadCamRight', 'LeftCamLeft', 'LeftCamRight', 'mode', 'RightCamLeft', 'RightCamRight', 'rpy', 'velocity', 'yawSpeed']
sensors = ["BellyCamLeft", "HeadCamLeft", "accelerometer", "footForce", "gyroscope", "LeftCamRight", "RightCamRight"]

# config for training
train_config_dict = {
    "epochs": 2,
    "batch_size": 8,
    "optimizer": "adam",
    "lr": 0.001,
    "momentum": 0.9,
    "dropout_rate": 0.2,
    "num_classes": 4,
    "use_wandb": True,
    "visualize_results": True,
    "train_log_interval": 200,
    "sensors": sensors,
    "dataset_path": dataset_path
}

In [ ]:
run_paths_dict = gen_run_dir(r"D:\git_repos\FA_DL_Modelle\runs\run_31_08__20_46_54")
start_logger(run_paths_dict["logs_path"], stream_log=True)

In [ ]:
# create dataset
transformed_dataset = FloorTypeDetectionDataset(
    dataset_path, sensors, mapping_filename, preprocessing_config_filename)

# get all possible config dicts for logging
label_mapping_dict = transformed_dataset.get_mapping_dict()
preprocessing_config_dict = transformed_dataset.get_preprocessing_config()

# split in train and test dataset
train_size = int(0.8 * len(transformed_dataset))
test_size = len(transformed_dataset) - train_size
ds_train, ds_test = torch.utils.data.random_split(
    transformed_dataset, [train_size, test_size])

In [ ]:
# define model, loss and optimizer
# model = LeNet_Like(train_config_dict["num_classes"])
model = VGG_Like(train_config_dict["num_classes"], train_config_dict["dropout_rate"])

In [ ]:
# ## multimodal models
# determine number of input features for all timeseries sensors
num_input_features_dict = {}
for sensor in sensors:
    if not "Cam" in sensor:
        _, (training_sample, _) = next(enumerate(transformed_dataset))
        num_input_features_dict[sensor] = training_sample[sensor].size()[0]
# define multimodal model
model = LeNet_Like_multimodal(train_config_dict["num_classes"], sensors, num_input_features_dict, train_config_dict["dropout_rate"])

In [ ]:
# training loop
trainer = Trainer(model, ds_train, ds_test, sensors,
                    train_config_dict, run_paths_dict)

In [ ]:
trainer.train()

In [ ]:
# optionally load instead of train the model
num = 2
load_path = os.path.join(run_paths_dict["model_ckpts"], f"{model._get_name()}_{num}.pt")
load_state_dict(model, load_path)

In [ ]:
print(model)

In [ ]:
#extract state dict
params_dict = model.state_dict()

weights = params_dict["classification_layers.0.weight"].numpy()
biases = params_dict["classification_layers.0.bias"]

In [ ]:
# needed infos about feature shape
flatten_length_cam_feature = 16 * 13 ** 2
flatten_length_IMU_feature = 16 * 9

# normalize weights
min_val = np.min(weights)
norm_weights = weights - min_val
max_val = np.max(norm_weights)
norm_weights = norm_weights / max_val

# extract weights for layers
weights = norm_weights
extracted_weights = []
current_pos = 0
for sensor in sensors:
    if "Cam" in sensor:
        new_pos = (current_pos+flatten_length_cam_feature)
        extracted_weights.append(weights[:, current_pos:new_pos])
        current_pos = new_pos
    else:
        new_pos = (current_pos+flatten_length_IMU_feature)
        extracted_weights.append(weights[:, current_pos:new_pos])
        current_pos = new_pos

In [ ]:
# create plot
rows = 3
columns =3
fig = plt.figure(figsize=(20, 20))
ax = []

# add all subplots
for index, cur_weight in enumerate(extracted_weights):
    # add next subplot and title
    ax.append(fig.add_subplot(rows, columns, index+1))
    ax[-1].set_title(sensors[index])
    plt.imshow(cur_weight.transpose(1,0), cmap="viridis", vmin=0, vmax = 1, interpolation="none")


In [ ]:
# complete plot (to big)
plt.imshow(test["classification_layers.3.weight"].numpy(), cmap="viridis", interpolation="nearest")

In [ ]:
evaluate(model, ds_train, sensors, train_config_dict)

In [ ]:
# store used config as final step of logging
store_used_config(run_paths_dict, label_mapping_dict, preprocessing_config_dict, train_config_dict)